In [1]:
# sys.path.append('/home/super/waterffle/playlist')

In [2]:
# print(sys.path)

In [3]:
# gpu 메모리 일부로 제한
# torch.cuda.set_per_process_memory_fraction(0.5,1)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from preprocessing import load_ausil_feature_dataset

In [4]:
cs, ns, labels = load_ausil_feature_dataset()

====== Get extracted feature by pre-trained CNN ======


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32000/32000 [00:22<00:00, 1443.49it/s]


In [5]:
# bsz * 10 개만 할당
part = 3200
cs = torch.tensor(cs[:part])
ns = torch.tensor(ns[:part])
labels = torch.tensor(labels[:part])

# cs = torch.tensor(cs)
# ns = torch.tensor(ns)
# labels = torch.tensor(labels)

In [7]:
cs.size(), ns.size()

(torch.Size([3200, 1, 2528]), torch.Size([3200, 1, 2528]))

In [22]:
ns = ns.transpose(1,2)
ns.size()

torch.Size([3200, 2528, 1])

In [23]:
cs.size(), ns.size()

(torch.Size([3200, 1, 2528]), torch.Size([3200, 2528, 1]))

In [20]:
cs[0].size()

torch.Size([1, 2528])

-----------------------------

# load the features from pre-trained model

# 1. just compute cosine similarity between cs and ns of the features (no train)

In [ ]:
class PCS(nn.Module):
    '''
    PLM based Cosine Similarity
    '''
    def __init__(self):
        super(PCS, self).__init__()
    
    def forward(self, x):
        ctxt_emb, cand_emb = x
        
        # score: cand_emb와 ctxt_emb 간 cosine similarity값 (반환값)
        scores = torch.sum(ctxt_emb * cand_emb, -1)
        print(f"scores : {scores.size()}")
        scores = scores.view(1, -1)
        return scores

# 2. PLM features -> transformer -> cosine similarity

In [ ]:
class PCT(nn.Module):
    '''
    PLM based Cosine Similarity 
    '''
    def __init__(self,
                 input_dim,
                 embed_dim,
                 attention_num_heads,
                 dropout,
                 activation='gelu'
                ):
        super(PCT, self).__init__()
        
        self.att_encoder_block = nn.ModuleList()
        
        # conv encoder를 거쳐나온 데이터의 dim과 embed_dim이 다를 경우 맞춰준다
        if input_dim != embed_dim:
            self.att_encoder_block.append(nn.Linear(input_dim, embed_dim))
        
        # SelfAttentionEncoder * num_att_layers
        for i in range(num_att_layers):
            self.att_encoder_block.append(
                torch.nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                 nhead=self.attention_num_heads, 
                                                 dim_feedforward=embed_dim*4, 
                                                 dropout=dropout, 
                                                 activation=activation)
            )
                            
        self.att_encoder_block = nn.Sequential(*self.att_encoder_block)

        
    
    def forward(self, x):
        ctxt_emb, cand_emb = x
        
        # score: cand_emb와 ctxt_emb 간 cosine similarity값 (반환값)
        scores = torch.sum(ctxt_emb * cand_emb, -1)
#         print(f"scores : {scores.size()}")
        scores = scores.view(1, -1)
        
        pred = self.att_encoder_block(scores)
    
        return pred

# 3. PLM features -> Poly Encoder

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.nn.init as init
from operator import itemgetter

class CnnFeatureExtractor(nn.Module):
    def __init__(self):
        super(CnnFeatureExtractor, self).__init__()
        self.globalpool = F.max_pool2d # can use max also

        self.layer1 = nn.Sequential(nn.Conv2d(1,16,kernel_size=3,padding=1),nn.BatchNorm2d(16),nn.GELU())
        self.layer2 = nn.Sequential(nn.Conv2d(16,16,kernel_size=3,padding=1),nn.BatchNorm2d(16),nn.GELU())
        self.layer3 = nn.MaxPool2d(2)

        self.layer4 = nn.Sequential(nn.Conv2d(16,32,kernel_size=3,padding=1),nn.BatchNorm2d(32),nn.GELU())
        self.layer5 = nn.Sequential(nn.Conv2d(32,32,kernel_size=3,padding=1),nn.BatchNorm2d(32),nn.GELU())
        self.layer6 = nn.MaxPool2d(2)

        self.layer7 = nn.Sequential(nn.Conv2d(32,64,kernel_size=3,padding=1),nn.BatchNorm2d(64),nn.GELU())
        self.layer8 = nn.Sequential(nn.Conv2d(64,64,kernel_size=3,padding=1),nn.BatchNorm2d(64),nn.GELU())
        self.layer9 = nn.MaxPool2d(2)

        self.layer10 = nn.Sequential(nn.Conv2d(64,128,kernel_size=3,padding=1),nn.BatchNorm2d(128),nn.GELU())
        self.layer11 = nn.Sequential(nn.Conv2d(128,128,kernel_size=3,padding=1),nn.BatchNorm2d(128),nn.GELU())
        self.layer12 = nn.MaxPool2d(2)

        self.layer13 = nn.Sequential(nn.Conv2d(128,256,kernel_size=3,padding=1),nn.BatchNorm2d(256),nn.GELU())
        self.layer14 = nn.Sequential(nn.Conv2d(256,256,kernel_size=3,padding=1),nn.BatchNorm2d(256),nn.GELU())
        self.layer15 = nn.MaxPool2d(2) #

        self.layer16 = nn.Sequential(nn.Conv2d(256,512,kernel_size=3,padding=1),nn.BatchNorm2d(512),nn.GELU())
        self.layer17 = nn.MaxPool2d(2) # 
        
        self.layer18 = nn.Sequential(nn.Conv2d(512,1024,kernel_size=2),nn.BatchNorm2d(1024),nn.GELU())

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out1 = self.layer18(out)
        
        out = self.globalpool(out1,kernel_size=out1.size()[2:])
        out = out.view(out.size(0),-1)
        return out #,out1

In [31]:
class WF(nn.Module):
    def __init__(self):
        super(WF, self).__init__()
        
        # context encoder 
        self.ctxt_encoder = CnnFeatureExtractor()
        
        # candidate encoder
        self.cand_encoder = CnnFeatureExtractor()
        
    def forward(self, x):
        cs, ns = x
        
        ctxt_emb = self.ctxt_encoder(cs)
        cand_emb = self.ctxt_encoder(ns)
        
        # score: cand_emb와 ctxt_emb 간 cosine similarity값 (반환값)
        scores = torch.sum(ctxt_emb * cand_emb, -1)
#         print(f"scores : {scores.size()}")
        scores = scores.view(1, -1)
        return scores

In [12]:
import logging

logger = logging.getLogger(__name__)

In [16]:
from sklearn.metrics import accuracy_score

--------------------------------------------

In [16]:
def eval_model(model, dataloader, criterion, device):
    # layer 중 dropout layer와 같이 학습 시에는 사용하는데 inference할 때는 사용하지 않는 경우를 구분해주기 위함
    model.eval()
    
    n_steps = 0
    eval_loss = 0    
    acc = 0

    # autograd engine을 끔(gradient 계산하지 않음) --> 메모리 사용량을 줄이고 연산 속도를 높이기 위함
    with torch.no_grad():
        # input은 이미 1차 임베딩은 되어 있는 것을 가져왔다고 가정한다.
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(dataloader):
            n_steps+=1
            batch_label = torch.LongTensor([torch.argmax(batch_label).item()]).to(device)
            scores = model(x=[batch_cs, batch_ns]).detach()
            
#             print(f'eval score : {scores}')
                
            loss = criterion(scores, batch_label)

            eval_loss += loss.item()
            
#             print(f'predict : {torch.argmax(scores, axis=1)}')
#             print(f'label  : {batch_label}')
            if torch.argmax(scores, axis=1) == batch_label:
                acc += 1
    if n_steps == 0:
        raise Exception('Eval Metric must have at least one example before it can be computed.')
    results = {
        'eval_accuracy': acc / n_steps,
        'eval_loss': eval_loss / n_steps
    }
    return results
    

In [29]:

import os
# from transformers.optimization import AdamW, get_linear_schedule_with_warmup

def train(model, raw_data, epochs, batch_size, output_dir, lr, device='cpu', betas=(0.9, 0.999), weight_decay: float = 0.01, val_ratio=0.2, fp16=False, fp16_opt_level='O1'):
    import datetime
    import time
    # fp16=True면 amp를 통한 mixed preicision training을 한다는 의미
    # 사용 조건 : Volta 이상의 nvidia 그래픽 카드(v100, rtx2080ti, 등)
    if fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)
    
    def train_batch(data, bsz, shuffle=True):
        cs_rep, ns_rep, labels = data
        length = cs_rep.size()[0]
        
        for i in range(0, length, bsz):
            cs_batch = cs_rep[i:i+bsz]
            ns_batch = ns_rep[i:i+bsz]
            label_batch = labels[i:i+bsz]
            
            if shuffle:
                indexes = torch.randperm(bsz)              
                yield [cs_batch[indexes], ns_batch[indexes], label_batch[indexes]]
            else:
                yield [cs_batch, ns_batch, label_batch]    
    now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    state_save_path = os.path.join(output_dir, '{}_{}_{}_pytorch_model.bin'.format('bi-encoder', 'mls', now))
        
    best_eval_loss = float('inf')
    best_test_loss = float('inf')
        
    cs, ns, labels = raw_data[0].to(device), raw_data[1].to(device), raw_data[2].to(device)

    # 재사용 가능한 임베딩들은 미리 뽑기
    trainset = None
    valset = None

    if val_ratio > 0:
        train_len = int((1-val_ratio)*len(labels))
        trainset = [cs[:train_len], ns[:train_len], labels[:train_len]]
        valset = [cs[train_len:], ns[train_len:], labels[train_len:]]
    else:
        trainset = raw_data

    eval_freq = (len(trainset[2]) / batch_size) // 3
    
    t_total = len(trainset[0])
    
#     optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=100, num_training_steps=t_total
#     )

    model.to(device)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)

    # Set the learning rate of each parameter group using a cosine annealing schedule,
    # https://pytorch.org/docs/1.8.0/optim.html?highlight=cosineannealingwarmrestarts#torch.optim.lr_scheduler.CosineAnnealingWarmRestarts
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    # stepRL
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=eval_freq, gamma=0.95)

        
    batch_labels = []
    scores = []
    global_step = 0
    #TODO : 나중에 tqdm으로 바꾸기
    for epoch in range(1, epochs+1):
        start_time = time.time()
        train_batches = train_batch(trainset, batch_size)
        
        total_loss = 0.0
        accuracy = 0.0
        local_step = 0
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(train_batches):
            model.train()
            
            # label
            batch_label = torch.LongTensor([torch.argmax(batch_label).item()]).to(device)
            
            # dot product
            scores = model([batch_cs, batch_ns])
#             print(f'scores : {scores}')

            # cross entropy
            loss = criterion(scores, batch_label)

            total_loss += loss.item()
            
            predict = torch.argmax(scores, axis=1)
#             print('predict : ',predict.item())
#             print(f'label : {batch_label.item()}\n')
#             if predict == torch.argmax(batch_label).item():
            if predict == batch_label:
                accuracy += 1
#                 print(f'-- correct : {accuracy}\n')
            
            if torch.isnan(loss):
                print(f'scores : {scores}')
                exit()
            
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            optimizer.step()
#             scheduler.step()

            # clear gradient
            optimizer.zero_grad()

            
#             model.zero_grad()
            
#             if global_step and global_step % eval_freq == 0:
#                 if valset is not None:
#                     val_batches = train_batch(valset, batch_size, shuffle=False)

#                 eval_results = eval_model(val_batches, criterion)
#                 scheduler.step(eval_results['eval_loss'])

#                 print(f'Global Step {global_step} Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
#                 if eval_results["eval_loss"] < best_eval_loss:
#                     best_eval_loss = eval_results['eval_loss']
#                     # save model.state_dict()
#                     torch.save(model.state_dict(), state_save_path)
#                     print(f'[Saving at] {state_save_path}')

            
            global_step += 1
            local_step += 1

            #print(f'batch {idx} | Loss {loss.item()}')
        
        total_loss = total_loss / local_step
        train_accuracy = accuracy / local_step
        print(f'\n[Epoch {epoch}] | train_cost {total_loss} | train_accuracy {train_accuracy} | time {time.time()-start_time}')
        
        if valset is not None:
            val_batches = train_batch(valset, batch_size, shuffle=False)
        eval_results = eval_model(model, val_batches, criterion, device)
        print(f'Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
        if eval_results["eval_loss"] < best_eval_loss:
            best_eval_loss = eval_results['eval_loss']
            # save model.state_dict()
            torch.save(model.state_dict(), state_save_path)
#             print(f'[Saving at] {state_save_path}')
        print('---------------------------------\n')
            

In [32]:
# RuntimeError: Expected 4-dimensional input for 4-dimensional weight [256, 1, 3, 3], but got 3-dimensional input of size [320, 40, 2800] instead
# tdata = [now_mfcc_list.to(1), next_mfcc_list.to(1), label_list.to(1)]
tdata = [cs, ns, labels]
# batch_size, C, H, W = 32, 1, 28, 28
# tdata = [torch.randn(batch_size, C, H, W).to(1),torch.randn(batch_size, C, H, W).to(1), torch.randn(batch_size).to(1)]

# Hyperparameters
epochs = 100 # epoch
batch_size=32 # batch size for training
lr=1e-5 # learning rate

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)
device='cuda:1'

model = WF()

train(model=model, 
      raw_data=tdata, 
      epochs=epochs, 
      batch_size=batch_size, 
      output_dir='.', 
      lr=lr,
      device=device,
      val_ratio=0.2, 
      fp16=False, 
      fp16_opt_level='O1')


[Epoch 1] | train_cost 551.3427799224853 | train_accuracy 0.0 | time 10.59098768234253
Eval loss : 229.72518892288207, Eval accuracy : 0.0
---------------------------------


[Epoch 2] | train_cost 100.51864422700615 | train_accuracy 0.375 | time 10.356552124023438
Eval loss : 253.44021186828613, Eval accuracy : 0.0
---------------------------------


[Epoch 3] | train_cost 25.5765392055353 | train_accuracy 0.65 | time 10.35309624671936
Eval loss : 272.48412556648253, Eval accuracy : 0.05
---------------------------------


[Epoch 4] | train_cost 6.261235316135935 | train_accuracy 0.8625 | time 10.347128868103027
Eval loss : 286.4800060272217, Eval accuracy : 0.0
---------------------------------


[Epoch 5] | train_cost 5.025358309364265 | train_accuracy 0.875 | time 10.359852313995361
Eval loss : 279.5222639083862, Eval accuracy : 0.05
---------------------------------


[Epoch 6] | train_cost 3.843585693819249 | train_accuracy 0.9375 | time 10.374258279800415
Eval loss : 294.686658


[Epoch 50] | train_cost 0.0 | train_accuracy 1.0 | time 10.44254207611084
Eval loss : 305.05349044799806, Eval accuracy : 0.0
---------------------------------


[Epoch 51] | train_cost 0.0 | train_accuracy 1.0 | time 10.431551456451416
Eval loss : 305.0271598815918, Eval accuracy : 0.0
---------------------------------


[Epoch 52] | train_cost 0.0 | train_accuracy 1.0 | time 10.43422245979309
Eval loss : 304.99860153198244, Eval accuracy : 0.0
---------------------------------


[Epoch 53] | train_cost 0.0 | train_accuracy 1.0 | time 10.444721698760986
Eval loss : 304.96956100463865, Eval accuracy : 0.0
---------------------------------


[Epoch 54] | train_cost 0.0 | train_accuracy 1.0 | time 10.44548487663269
Eval loss : 304.9391227722168, Eval accuracy : 0.0
---------------------------------


[Epoch 55] | train_cost 0.0 | train_accuracy 1.0 | time 10.444405555725098
Eval loss : 304.9072319030762, Eval accuracy : 0.0
---------------------------------


[Epoch 56] | train_cost 0.0

KeyboardInterrupt: 